In [ ]:
import os
os.chdir('/mmsegmentation')

In [ ]:
#To test lens soiling

import os
import numpy as np
import pandas as pd
import mmcv
import torch
from tqdm import tqdm
from mmseg.apis import init_model
from mmseg.utils import register_all_modules
from mmseg.evaluation import IoUMetric
from mmcv.transforms import Compose
from mmengine.registry import TRANSFORMS
from mmseg.datasets.transforms import PackSegInputs
import cv2



# Gaussian Blur
class RandomApplyGaussianBlur:
    def __init__(self, prob=0.5, kernel_size=(5, 5), sigma=0):
        self.prob = prob
        self.kernel_size = kernel_size
        self.sigma = sigma

    def __call__(self, results):
        apply = np.random.rand() < self.prob
        if apply:
            img = results['img']
            blurred = cv2.GaussianBlur(img, self.kernel_size, self.sigma)
            results['img'] = blurred
        results.setdefault('metainfo', {})
        results['metainfo']['gaussian_blur_applied'] = apply
        return results

# Gaussian Noise
class RandomApplyGaussianNoise:
    def __init__(self, prob=0.5, mean=0, std=10):
        self.prob = prob
        self.mean = mean
        self.std = std

    def __call__(self, results):
        apply = np.random.rand() < self.prob
        if apply:
            img = results['img'].astype(np.float32)
            noise = np.random.normal(self.mean, self.std, img.shape).astype(np.float32)
            noisy_img = np.clip(img + noise, 0, 255).astype(np.uint8)
            results['img'] = noisy_img
        results.setdefault('metainfo', {})
        results['metainfo']['gaussian_noise_applied'] = apply
        return results
        

class CustomPackSegInputs(PackSegInputs):
    def transform(self, results):
        packed = super().transform(results)
        metainfo = results.get('metainfo', {})
        extra_keys = ['gaussian_blur_applied', 'gaussian_noise_applied', 'flip']
        for key in extra_keys:
            if key in metainfo:
                packed['data_samples'].set_metainfo({key: metainfo[key]})
        return packed


register_all_modules()


config_file = 'configs/MobileNet/Mobilenet_test.py'
checkpoint_file = 'work_dirs/Mobilnet/best_checkpoint.pth'



model = init_model(config_file, checkpoint_file, device='cuda:0')

if model.dataset_meta is None:
    model.dataset_meta = {
        'classes': ['background', 'road'],
        'palette': [[127, 127, 127], [0, 0, 255]]
    }


img_dir = 'automine1d_distortion/lente_sucio/img'
gt_dir = 'automine1d_distortion/lente_sucio/ann'


img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])
gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.png')])

pipeline_cfg = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    RandomApplyGaussianBlur(prob=0.5, kernel_size=(5, 5), sigma=1),
    RandomApplyGaussianNoise(prob=0.5, mean=0, std=10),
    CustomPackSegInputs()
]

pipeline = Compose([TRANSFORMS.build(t) if isinstance(t, dict) else t for t in pipeline_cfg])

num_repetitions = 10
results_all = []

for repetition in range(num_repetitions):
    print(f"🔁 Repetición {repetition + 1}")
    for img_file, gt_file in tqdm(zip(img_files, gt_files), desc=f"Repetición {repetition+1}", total=len(img_files)):
        img_path = os.path.join(img_dir, img_file)
        gt_path = os.path.join(gt_dir, gt_file)

        data = dict(
            img_path=img_path,
            seg_map_path=gt_path,
            reduce_zero_label=False,
            seg_fields=[]
        )

        aug_data = pipeline(data)
        flip_applied = aug_data['data_samples'].metainfo.get('flip', False)
        blur_applied = aug_data['data_samples'].metainfo.get('gaussian_blur_applied', False)
        noise_applied = aug_data['data_samples'].metainfo.get('gaussian_noise_applied', False)

        aug_batch = {
            'inputs': aug_data['inputs'].unsqueeze(0),
            'data_samples': [aug_data['data_samples']]
        }

        result = model.test_step(aug_batch)[0]
        pred_mask_np = result.pred_sem_seg.data.squeeze().cpu().numpy().astype(np.uint8)

        gt_mask_np = mmcv.imread(gt_path, flag='unchanged')
        if len(gt_mask_np.shape) == 3:
            gt_mask_np = gt_mask_np[:, :, 0]

        pred_tensor = torch.from_numpy(pred_mask_np).long()
        gt_tensor = torch.from_numpy(gt_mask_np).long()

        evaluator = IoUMetric(iou_metrics=['mIoU', 'mDice', 'mFscore'], collect_device='cpu')
        evaluator.dataset_meta = model.dataset_meta
        evaluator.process([], [{
            'pred_sem_seg': {'data': pred_tensor},
            'gt_sem_seg': {'data': gt_tensor}
        }])
        metrics = evaluator.evaluate(size=1)

        row = {
            'Repetition': repetition + 1,
            'ImageName': img_file,
            'mIoU': metrics.get('mIoU', np.nan),
            'mDice': metrics.get('mDice', np.nan),
            'mFscore': metrics.get('mFscore', np.nan),
            'Flip': flip_applied,            
            'GaussianBlur': blur_applied,
            'GaussianNoise': noise_applied
        }

        results_all.append(row)

df = pd.DataFrame(results_all)

# Añadir promedio y desviación estándar
mean_row = df.mean(numeric_only=True)
mean_row['Repetition'] = 'Promedio'
std_row = df.std(numeric_only=True)
std_row['Repetition'] = 'Desviación estándar'
df = pd.concat([df, pd.DataFrame([mean_row, std_row])], ignore_index=True)

output_path = '/work_dirs/Mobilnet/Mobilnet_lr0.001_Clahe_photo/result_data_lentesucio.xlsx'
df.to_excel(output_path, index=False)

df.to_excel(output_path, index=False)

# Mostrar resumen 
summary_df = df[df['Repetition'].isin(['Promedio', 'Desviación estándar'])]
from IPython.display import display
display(summary_df)

print(f" Detalles guardados en {output_path}")

#To test sun glare

import os
import numpy as np
import pandas as pd
import mmcv
import torch
from tqdm import tqdm
from mmseg.apis import init_model
from mmseg.utils import register_all_modules
from mmseg.evaluation import IoUMetric
from mmcv.transforms import Compose
from mmengine.registry import TRANSFORMS
from mmseg.datasets.transforms import PackSegInputs


# Gaussian Blur
class RandomApplyGaussianBlur:
    def __init__(self, prob=0.5, kernel_size=(5, 5), sigma=0):
        self.prob = prob
        self.kernel_size = kernel_size
        self.sigma = sigma

    def __call__(self, results):
        apply = np.random.rand() < self.prob
        if apply:
            img = results['img']
            blurred = cv2.GaussianBlur(img, self.kernel_size, self.sigma)
            results['img'] = blurred
        results.setdefault('metainfo', {})
        results['metainfo']['gaussian_blur_applied'] = apply
        return results

# Gaussian Noise
class RandomApplyGaussianNoise:
    def __init__(self, prob=0.5, mean=0, std=10):
        self.prob = prob
        self.mean = mean
        self.std = std

    def __call__(self, results):
        apply = np.random.rand() < self.prob
        if apply:
            img = results['img'].astype(np.float32)
            noise = np.random.normal(self.mean, self.std, img.shape).astype(np.float32)
            noisy_img = np.clip(img + noise, 0, 255).astype(np.uint8)
            results['img'] = noisy_img
        results.setdefault('metainfo', {})
        results['metainfo']['gaussian_noise_applied'] = apply
        return results


class CustomPackSegInputs(PackSegInputs):
    def transform(self, results):
        packed = super().transform(results)
        metainfo = results.get('metainfo', {})
        extra_keys = ['gaussian_blur_applied', 'gaussian_noise_applied', 'flip']
        for key in extra_keys:
            if key in metainfo:
                packed['data_samples'].set_metainfo({key: metainfo[key]})
        return packed


register_all_modules()

model = init_model(config_file, checkpoint_file, device='cuda:0')

if model.dataset_meta is None:
    model.dataset_meta = {
        'classes': ['background', 'road'],
        'palette': [[127, 127, 127], [0, 0, 255]]
    }

img_dir = '/automine1d_distortion/haz_luz/img'
gt_dir = '/automine1d_distortion/haz_luz/ann'

img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])
gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.png')])

pipeline_cfg = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    RandomApplyGaussianBlur(prob=0.5, kernel_size=(5, 5), sigma=1),
    RandomApplyGaussianNoise(prob=0.5, mean=0, std=10),
    CustomPackSegInputs()
]
pipeline = Compose([TRANSFORMS.build(t) if isinstance(t, dict) else t for t in pipeline_cfg])

num_repetitions = 15
results_all = []

for repetition in range(num_repetitions):
    print(f"🔁 Repetición {repetition + 1}")
    for img_file, gt_file in tqdm(zip(img_files, gt_files), desc=f"Repetición {repetition+1}", total=len(img_files)):
        img_path = os.path.join(img_dir, img_file)
        gt_path = os.path.join(gt_dir, gt_file)

        data = dict(
            img_path=img_path,
            seg_map_path=gt_path,
            reduce_zero_label=False,
            seg_fields=[]
        )

        aug_data = pipeline(data)
        flip_applied = aug_data['data_samples'].metainfo.get('flip', False)
        blur_applied = aug_data['data_samples'].metainfo.get('gaussian_blur_applied', False)
        noise_applied = aug_data['data_samples'].metainfo.get('gaussian_noise_applied', False)





        aug_batch = {
            'inputs': aug_data['inputs'].unsqueeze(0),
            'data_samples': [aug_data['data_samples']]
        }

        result = model.test_step(aug_batch)[0]
        pred_mask_np = result.pred_sem_seg.data.squeeze().cpu().numpy().astype(np.uint8)

        gt_mask_np = mmcv.imread(gt_path, flag='unchanged')
        if len(gt_mask_np.shape) == 3:
            gt_mask_np = gt_mask_np[:, :, 0]

        pred_tensor = torch.from_numpy(pred_mask_np).long()
        gt_tensor = torch.from_numpy(gt_mask_np).long()

        evaluator = IoUMetric(iou_metrics=['mIoU', 'mDice', 'mFscore'], collect_device='cpu')
        evaluator.dataset_meta = model.dataset_meta
        evaluator.process([], [{
            'pred_sem_seg': {'data': pred_tensor},
            'gt_sem_seg': {'data': gt_tensor}
        }])
        metrics = evaluator.evaluate(size=1)

        row = {
            'Repetition': repetition + 1,
            'ImageName': img_file,
            'mIoU': metrics.get('mIoU', np.nan),
            'mDice': metrics.get('mDice', np.nan),
            'mFscore': metrics.get('mFscore', np.nan),
            'Flip': flip_applied,
            'GaussianBlur': blur_applied,
            'GaussianNoise': noise_applied
        }

        results_all.append(row)

df = pd.DataFrame(results_all)

# Añadir promedio y desviación estándar
mean_row = df.mean(numeric_only=True)
mean_row['Repetition'] = 'Promedio'
std_row = df.std(numeric_only=True)
std_row['Repetition'] = 'Desviación estándar'
df = pd.concat([df, pd.DataFrame([mean_row, std_row])], ignore_index=True)

output_path = './work_dirs/Mobilnet/Mobilnet_lr0.001_Clahe_photo/result_data_haz_luz.xlsx'
df.to_excel(output_path, index=False)

# Mostrar resumen 
summary_df = df[df['Repetition'].isin(['Promedio', 'Desviación estándar'])]
from IPython.display import display
display(summary_df)

print(f" Detalles guardados en {output_path}")